In [5]:

#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os.path
import pandas as pd
import sys
import tensorflow as tf

from IPython.display import display
from IPython.display import Image

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras import regularizers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

from xyz_S2integralsDeltaT_models import *
from helpers import *
from pax_utils import s1s2_utils

print (tf.__version__)

v_drift = 1.44e-4


1.4.1


In [6]:

#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

nEventsTrain = 100000
file_all     = "../../data_lfs/merged_all_200k_1803261941.pkl"

df_all   = pd.read_pickle(file_all)
df_all   = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all   = df_all[0:nEventsTrain][:]

df_all['intr_depth'] = v_drift*df_all['intr_drift_time'] 
#df_all['intr_depth'] = -1*v_drift*df_all['intr_drift_time'] 
#df_all['depth']      *= -1


#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

df_train_truth = df_all[:][['x', 'y', 'depth']]
df_train_input = df_all[:][['intr_depth'] + s1s2_utils.getS2integralsDataFrameColumns()]
#df_train_input = df_all[:][['intr_drift_time'] + s1s2_utils.getS2integralsDataFrameColumns()]


#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

checkTrainingData(df_train_input, df_train_truth)




Input shape: (100000, 128)
Truth shape: (100000, 3)

Input DataFrame: 


,intr_depth,s2_area_000,s2_area_001,s2_area_002,s2_area_003,s2_area_004,s2_area_005,s2_area_006,s2_area_007,s2_area_008,...,s2_area_117,s2_area_118,s2_area_119,s2_area_120,s2_area_121,s2_area_122,s2_area_123,s2_area_124,s2_area_125,s2_area_126
0,57.72816,1.715228,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,6.564991,0.0,0.400128,2.742959,3.460256,8.159419,38.424745,15.890681,1.354377,10.354574
1,58.00032,0.912039,0.0,0.0,1.545933,0.000000,0.000000,2.568431,0.000000,2.604376,...,3.208600,0.0,0.000000,5.596787,26.363896,126.844246,55.087875,8.130994,8.979807,13.682349
2,34.46640,0.000000,0.0,0.0,0.000000,2.104002,0.000000,1.731047,2.784006,0.000000,...,3.487908,0.0,1.534094,0.000000,1.919893,4.920848,0.505347,2.435258,0.506562,4.203870
3,64.68624,6.140486,0.0,0.0,9.062252,5.621141,1.789005,1.802042,0.000000,0.869444,...,0.000000,0.0,0.918565,0.619911,0.000000,1.230013,0.788772,1.179353,2.046051,0.000000
4,51.80544,4.333349,0.0,0.0,0.000000,1.313044,0.000000,0.613111,0.000000,1.061757,...,12.939582,0.0,3.896350,2.259100,3.023539,0.855252,7.456719,1.257868,4.551362,5.000204



Truth DataFrame:


,x,y,depth
0,2.626434,-17.960824,57.456815
1,10.907536,-6.312277,57.780505
2,24.631697,-12.763601,34.074937
3,2.726006,37.749081,64.494053
4,-18.078293,-23.212466,51.510242


In [7]:

#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

lst_models = [
    bargeModel_xyz()
]



In [8]:

#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

for iModel in range(0, len(lst_models)):
    
    #--------------------------------------------------------------------------
    #--------------------------------------------------------------------------

    model, model_name = lst_models[iModel]
    
    
    #--------------------------------------------------------------------------
    #--------------------------------------------------------------------------

    epochs     = 20
    batch_size = 64
    verbose    = 1
    
    
    #--------------------------------------------------------------------------
    # Fit Model
    #--------------------------------------------------------------------------
    
    print()
    print("Model Summary:")
    model.summary()
    print()
    model.fit(df_train_input, df_train_truth, epochs=epochs, batch_size=batch_size, verbose=verbose)
    print("Training Complete!")

        
    #--------------------------------------------------------------------------
    # Save Model
    #--------------------------------------------------------------------------

    folder   = "models" + "/"    
    name     = 'model' + '_' + model_name + '_' + 'elu' #model.activation
    name_h5  = folder + name + ".h5"
    name_png = folder + name + ".png"

    model.save(name_h5, overwrite=True)
        
   
    #--------------------------------------------------------------------------
    #--------------------------------------------------------------------------

    plot_model(model, to_file=name_png, show_layer_names=True, show_shapes=True)
    
    Image(filename=name_png) 
    

    #--------------------------------------------------------------------------
    #--------------------------------------------------------------------------
    
    continue
    
    


Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 509)               65661     
_________________________________________________________________
dropout_2 (Dropout)          (None, 509)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 1530      
Total params: 83,703
Trainable params: 83,703
Non-trainable params: 0
_________________________________________________________________

Epoch 1/20
100000/100000 [==============================] - 22s 225us/step - loss: 20.5477 - acc: 0.9694
Epoch 2/20
100000/100000 [==============================] - 20s 200us/step - loss: 3.4392 - acc: 0.9881
Epoch 3/20
100000/100000 [==================